In [ ]:
!pip install transformers datasets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# ***Preprocessing***

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

data = pd.read_csv('bias.csv')

data = data[['docbody', 'article_bias']].dropna()
data.rename(columns={'docbody': 'text', 'article_bias': 'label'}, inplace=True)

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_idx, temp_idx in stratified_split.split(data['text'], data['label']):
    train_texts = data['text'].iloc[train_idx]
    train_labels = data['label'].iloc[train_idx]
    temp_texts = data['text'].iloc[temp_idx]
    temp_labels = data['label'].iloc[temp_idx]

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for val_idx, test_idx in stratified_split.split(temp_texts, temp_labels):
    val_texts = temp_texts.iloc[val_idx]
    val_labels = temp_labels.iloc[val_idx]
    test_texts = temp_texts.iloc[test_idx]
    test_labels = temp_labels.iloc[test_idx]

print("Training set class distribution:")
print(train_labels.value_counts())
print("Validation set class distribution:")
print(val_labels.value_counts())
print("Test set class distribution:")
print(test_labels.value_counts())


Training set class distribution:
label
3    64
2    50
1    23
4    13
Name: count, dtype: int64
Validation set class distribution:
label
3    14
2    11
1     4
4     3
Name: count, dtype: int64
Test set class distribution:
label
3    14
2    11
1     5
4     3
Name: count, dtype: int64


In [ ]:
from datasets import Dataset

unique_labels = sorted(train_labels.unique())
label_map = {label: i for i, label in enumerate(unique_labels)}

train_labels = train_labels.map(label_map)
val_labels = val_labels.map(label_map)
test_labels = test_labels.map(label_map)

train_dataset = Dataset.from_dict({'text': train_texts.tolist(), 'label': train_labels.tolist()})
val_dataset = Dataset.from_dict({'text': val_texts.tolist(), 'label': val_labels.tolist()})
test_dataset = Dataset.from_dict({'text': test_texts.tolist(), 'label': test_labels.tolist()})


In [ ]:
from transformers import AutoTokenizer

model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples['text'], truncation=True, padding='max_length', max_length=512
    )

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

tokenized_train = tokenized_train.map(lambda x: {'label': int(x['label'])})
tokenized_val = tokenized_val.map(lambda x: {'label': int(x['label'])})
tokenized_test = tokenized_test.map(lambda x: {'label': int(x['label'])})

tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


...

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

# **Training Parameters**

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(label_map)
)

training_args = TrainingArguments(
    output_dir="./deberta_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./deberta_logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,
    report_to="none",
)


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# **Training and Evaluation**

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted", zero_division=1),
        "precision": precision_score(labels, predictions, average="weighted", zero_division=1),
        "recall": recall_score(labels, predictions, average="weighted", zero_division=1),
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


<ipython-input-6-2ef93dbf3276>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

val_results = trainer.evaluate(eval_dataset=tokenized_val)
print("Validation Set Results:", val_results)

test_results = trainer.evaluate(eval_dataset=tokenized_test)
print("Test Set Results:", test_results)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.252850,0.437500,0.266304,0.753906,0.437500
2,No log,1.216131,0.437500,0.266304,0.753906,0.437500
3,No log,1.210692,0.437500,0.266304,0.753906,0.437500
4,No log,1.204045,0.437500,0.266304,0.753906,0.437500
5,No log,1.198411,0.437500,0.266304,0.753906,0.437500


Validation Set Results: {'eval_loss': 1.2528495788574219, 'eval_accuracy': 0.4375, 'eval_f1': 0.266304347826087, 'eval_precision': 0.75390625, 'eval_recall': 0.4375, 'eval_runtime': 0.8803, 'eval_samples_per_second': 36.353, 'eval_steps_per_second': 4.544, 'epoch': 5.0}
Test Set Results: {'eval_loss': 1.2632273435592651, 'eval_accuracy': 0.42424242424242425, 'eval_f1': 0.2527401676337846, 'eval_precision': 0.7557392102846647, 'eval_recall': 0.42424242424242425, 'eval_runtime': 1.1733, 'eval_samples_per_second': 28.127, 'eval_steps_per_second': 4.262, 'epoch': 5.0}


finetuning